In [1]:
# Imports
import os
import base64
import requests
#from abc import ABC
import pandas as pd
from pandas.io.json import json_normalize
import geopandas as gpd

# Define aWhere API key and secret
api_key = os.environ.get('AWHERE_API_KEY')
api_secret = os.environ.get('AWHERE_API_SECRET')

# Level 1: Base Class - AWhereAPI (make this abstract base class?)

In [2]:
class AWhereAPI():
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None):
        # Define authorization information
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_64_encoded_secret_key = self.encode_secret_and_key(
            self.api_key, self.api_secret)
        self.auth_token = self.get_oauth_token(self.base_64_encoded_secret_key)

    def encode_secret_and_key(self, key, secret):
        """
        Docs:
            http://developer.awhere.com/api/authentication
        Returns:
            Returns the base64-encoded {key}:{secret} combination, seperated by a colon.
        """
        # Base64 Encode the Secret and Key
        key_secret = f"{key}:{secret}"

        encoded_key_secret = base64.b64encode(
            bytes(key_secret, 'utf-8')).decode('ascii')

        return encoded_key_secret

    def get_oauth_token(self, encoded_key_secret):
        """
        Demonstrates how to make a HTTP POST request to obtain an OAuth Token

        Docs:
            http://developer.awhere.com/api/authentication

        Returns:
            The access token provided by the aWhere API
        """
        # Define authorization parameters
        auth_url = 'https://api.awhere.com/oauth/token'

        auth_headers = {
            "Authorization": f"Basic {encoded_key_secret}",
            'Content-Type': 'application/x-www-form-urlencoded'
        }

        body = "grant_type=client_credentials"

        # Perform HTTP request for OAuth Token
        response = requests.post(
            auth_url, headers=auth_headers, data=body)

        # Return access token
        return response.json()['access_token']

# Level 2: Subclass - Weather (make this abstract class?)

In [3]:
class Weather(AWhereAPI):
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None, api_url=None):
        super(Weather, self).__init__(api_key, api_secret,
                                      base_64_encoded_secret_key, auth_token)

        self.api_url = 'https://api.awhere.com/v2/weather'

    def get_data():
        pass
        
    @staticmethod
    def extract_data():
        pass

    @staticmethod
    def clean_data(df, lon_lat_cols, drop_cols, name_map):
        """Converts dataframe to geodataframe,
        drops unnecessary columns, and renames
        columns.

        Parameters
        ----------
        df : dataframe
            Input dataframe.

        lon_lat_cols : list
            List containing the column name for longitude (list[0])
            and latitude (list[1]) attributes.

        drop_cols : list (of str)
            List of column names to be dropped.

        name_map : dict
            Dictionaty mapping old columns names (keys)
            to new column names (values).

        Returns
        -------
        gdf : geodataframe
            Cleaned geodataframe.

        Example
        -------
        """
        # Define CRS (EPSG 4326) - make this a parameter?
        crs = {'init': 'epsg:4326'}

        # Rename index - possibly as option, or take care of index prior?
        #df.index.rename('date_rename', inplace=True)

        # Create copy of input dataframe; prevents altering the original
        df_copy = df.copy()

        # Convert to geodataframe
        gdf = gpd.GeoDataFrame(
            df_copy, crs=crs, geometry=gpd.points_from_xy(
                df[lon_lat_cols[0]],
                df[lon_lat_cols[1]])
        )

        # Add lat/lon columns to drop columns list
        drop_cols += lon_lat_cols

        # Drop columns
        gdf.drop(columns=drop_cols, axis=1, inplace=True)

        # Rename columns
        gdf.rename(columns=name_map, inplace=True)

        # Return cleaned up geodataframe
        return gdf
    
#     @classmethod
#     def api_to_gdf(): # (cls, api_object, kwargs=None)
#         pass
    
    @classmethod
    def api_to_gdf(cls, api_object, kwargs=None):
        """kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists
        
        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        api_data_df =  cls.extract_data(api_data_json)

        api_data_gdf = cls.clean_data(
            api_data_df,
            cls.coord_cols,
            cls.drop_cols,
            cls.rename_map
        )

        return api_data_gdf

# Level 2: Subclass - Fields

In [1]:
class Fields(AWhereAPI):
    def __init__(self, api_key, api_secret, base_64_encoded_secret_key=None, auth_token=None, api_url=None):
        super(Fields, self).__init__(api_key, api_secret,
                                     base_64_encoded_secret_key, auth_token)

        self.api_url = 'https://api.awhere.com/v2/fields'

    # Modify this to return all fields into a dataframe?
    def get(self, field_id=None, limit=10, offset=0):
        """
        Performs a HTTP GET request to obtain all Fields you've created on your aWhere App.

        Docs:
            http://developer.awhere.com/api/reference/fields/get-fields
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        if field_id:
            # Perform the HTTP request to obtain a specific field
            fields_response = requests.get(f"{self.api_url}/{field_id}", headers=auth_headers)
            
            message = fields_response.json()
            
        else:
            # Perform the HTTP request to obtain a list of all fields
            fields_response = requests.get(
                f"{self.api_url}?limit={limit}&offset={offset}", headers=auth_headers)

            responseJSON = fields_response.json()

        
            # Display the count of Fields for the user account
            print(
                f"You have {len(responseJSON['fields'])} fields shown on this page.")

            # Iterate over the fields and display their name and ID
            print('#  Field Name \t\t Field ID')
            print('-------------------------------------------')
            count = 0
            for field in responseJSON["fields"]:
                count += 1
                print(f"{count}. {field['name']} \t {field['id']}\r")

            message = print("\nFields listed above.")
                              
        return message

    def create(self, field_id, field_name, farm_id, center_latitude, center_longitude, acres):
        """
        Performs a HTTP POST request to create and add a Field to your aWhere App.AWhereAPI, based on user input

        Docs:
            http://developer.awhere.com/api/reference/fields/create-field
        """
        field_body = {
            'id': field_id,
            'name': field_name,
            'farmId': farm_id,
            'centerPoint': {
                'latitude': center_latitude,
                'longitude': center_longitude
            },
            'acres': acres
        }

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {str(self.auth_token)}",
            "Content-Type": 'application/json'
        }

        # Perform the POST request to create Field
        print('Attempting to create new field....\n')
        response = requests.post(
            self.api_url, headers=auth_headers, json=field_body)

        return response.json()

    def update(self, field_id, name=None, farm_id=None):
        """Update the name and/or farm id for a field.
        """
        if not (name or farm_id):
            field_body = [{
                "op": "replace",
                "path": "/name",
                "value": name
            }, {
                "op": "replace",
                "path": "/farmId",
                "value": farm_id
            }]

        elif name and not farm_id:
            field_body = [{
                "op": "replace",
                "path": "/name",
                "value": name
            }]

        elif farm_id and not name:
            field_body = [{
                "op": "replace",
                "path": "/farmId",
                "value": farm_id
            }]

        elif name and farm_id:
            field_body = [{
                "op": "replace",
                "path": "/name",
                "value": name
            }, {
                "op": "replace",
                "path": "/farmId",
                "value": farm_id
            }]

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {str(self.auth_token)}",
        }

        # Perform the HTTP request to update field information
        response = requests.patch(
            f"{self.api_url}/{field_id}", headers=auth_headers, json=field_body)

        return response.json()

    def delete(self, field_id):
        """
        Performs a HTTP DELETE request to delete a Field from your aWhere App.
        Docs: http://developer.awhere.com/api/reference/fields/delete-field
        Args:
            field_id: The field to be deleted
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}",
            "Content-Type": 'application/json'
        }

        # Perform the POST request to Delete the Field
        response = requests.delete(
            f"{self.api_url}/{field_id}", headers=auth_headers)

        message = f"Deleted field: {field_id}" if response.status_code == 204 else f"Could not delete field."

        return print(message)

NameError: name 'AWhereAPI' is not defined

## Level 3: Sub-sub-class - WeatherLocation (make this abstract class?)

In [5]:
class WeatherLocation(Weather):
    
    def __init__(self, api_key, api_secret,
                 base_64_encoded_secret_key=None, auth_token=None, api_url=None):

        super(WeatherLocation, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.api_url = f"{self.api_url}/locations"

## Level 3: Sub-sub-class - WeatherField (make this an abstract class?)

In [6]:
class WeatherField(Weather):
    
    def __init__(self, api_key, api_secret,
                 base_64_encoded_secret_key=None, auth_token=None, api_url=None):

        super(WeatherField, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.api_url = f"{self.api_url}/fields"

## Level 3: Sub-sub-class - Field

In [7]:
class Field(Fields):
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None, auth_token=None, api_url=None):
        super(Field, self).__init__(api_key, api_secret,
                                    base_64_encoded_secret_key, auth_token)

        self.field_id = field_id
        self.api_url = f'https://api.awhere.com/v2/fields/{self.field_id}'

    def get(self):
        """
        Performs a HTTP GET request to obtain all Fields you've created on your aWhere App.

        Docs:
            http://developer.awhere.com/api/reference/fields/get-fields
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the field information
        field_response = requests.get(f"{self.api_url}",
                                      headers=auth_headers)

        #responseJSON = fields_response.json()

        return field_response.json()

    def create(self):
        pass

    def update(self, name=None, farm_id=None):
        """Update the name and/or farm id for a field.
        """
        if not (name or farm_id):
            field_body = [{
                "op": "replace",
                "path": "/name",
                "value": name
            }, {
                "op": "replace",
                "path": "/farmId",
                "value": farm_id
            }]

        elif name and not farm_id:
            field_body = [{
                "op": "replace",
                "path": "/name",
                "value": name
            }]

        elif farm_id and not name:
            field_body = [{
                "op": "replace",
                "path": "/farmId",
                "value": farm_id
            }]

        elif name and farm_id:
            field_body = [{
                "op": "replace",
                "path": "/name",
                "value": name
            }, {
                "op": "replace",
                "path": "/farmId",
                "value": farm_id
            }]

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {str(self.auth_token)}",
        }

        # Perform the HTTP request to update field information
        response = requests.patch(
            f"{self.api_url}", headers=auth_headers, json=field_body)

        return response.json()

    def delete(self):
        """
        Performs a HTTP DELETE request to delete a Field from your aWhere App.
        Docs: http://developer.awhere.com/api/reference/fields/delete-field
        Args:
            field_id: The field to be deleted
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}",
            "Content-Type": 'application/json'
        }

        # Perform the POST request to Delete the Field
        response = requests.delete(f"{self.api_url}", headers=auth_headers)

        message = f"Deleted field: {self.field_id}" if response.status_code == 204 else f"Could not delete field."

        return print(message)

## Level 4: Sub-sub-sub-class - WeatherLocationNorms

In [8]:
class WeatherLocationNorms(WeatherLocation):

    # Class variables for clean_data() function
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'meanTemp.units', 'maxTemp.units',
        'minTemp.units', 'precipitation.units', 'solar.units',
        'dailyMaxWind.units', 'averageWind.units'
    ]

    rename_map = {
        'meanTemp.average': 'mean_temp_avg_cels',
        'meanTemp.stdDev': 'mean_temp_std_dev_cels',
        'maxTemp.average': 'max_temp_avg_cels',
        'maxTemp.stdDev': 'max_temp_std_dev_cels',
        'minTemp.average': 'min_temp_avg_cels',
        'minTemp.stdDev': 'min_temp_std_dev_cels',
        'precipitation.average': 'precip_avg_mm',
        'precipitation.stdDev': 'precip_std_dev_mm',
        'solar.average': 'solar_avg_w_h_per_m2',
        'minHumidity.average': 'min_humiduty_avg_%',
        'minHumidity.stdDev': 'min_humidity_std_dev_%',
        'maxHumidity.average': 'max_humiduty_avg_%',
        'maxHumidity.stdDev': 'max_humidity_std_dev_%',
        'dailyMaxWind.average': 'daily_max_wind_avg_m_per_sec',
        'dailyMaxWind.stdDev': 'daily_max_wind_std_dev_m_per_sec',
        'averageWind.average': 'average_wind_m_per_sec',
        'averageWind.stdDev': 'average_wind_std_dev_m_per_sec'
    }

    # Define lat/lon when intitializing class; no need to repeat for lat/lon
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherLocationNorms, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/norms"

    def get_data(self, start_day='01-01', end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 10-year historical norms.

        Docs:
            http://developer.awhere.com/api/reference/weather/norms

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the norms.

        Example
        -------
        """
        """# Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        response = requests.get(
            f"{self._weather_url}/{field_id}/norms/{start_day}",
            headers=auth_headers)

        # Return the norms
        return response.json()"""

        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}?limit=10&offset={offset}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(historic_norms):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere historic norm data.

        Handles both single-day norms and multiple days.

        Parameters
        ----------
        historic_norms : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        historic_norms_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in norms
        if historic_norms.get('norms'):
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms.get('norms'))

        # Single-day norm
        else:
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms)

        # Set day as index
        historic_norms_df.set_index('day', inplace=True)

        # Drop unnecessary columns
        historic_norms_df.drop(
            columns=['_links.self.href'],
            axis=1, inplace=True)

        # Return dataframe
        return historic_norms_df

## Level 4: Sub-sub-sub-class - WeatherLocationObserved

In [9]:
class WeatherLocationObserved(WeatherLocation):
    # Class variables for clean_data() function
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units'
    ]

    rename_map = {
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.amount': 'precip_amount_mm',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.morningMax': 'wind_morning_max_m_per_sec',
        'wind.dayMax': 'wind_day_max_m_per_sec',
        'wind.average': 'wind_avg_m_per_sec',
    }

    def __init__(self, api_key, api_secret, latitude, longitude,
                 base_64_encoded_secret_key=None, auth_token=None, api_url=None):

        super(WeatherLocationObserved, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/observations"

    def get_data(self, start_day=None, end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 7-day observed weather.

        Docs:
            http://developer.awhere.com/api/reference/weather/observations

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the observed weather.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self.api_url}?limit=10&offset={offset}"
        url_start_date = f"{self.api_url}/{start_day}"
        url_end_date = f"{self.api_url}/{end_day}"
        url_both_dates = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Perform the HTTP request to obtain the norms for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return the observed
        return response.json()

    @staticmethod
    def extract_data(observed_weather):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere observed weather data.

        Parameters
        ----------
        observed_weather : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        observed_weather_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in observed
        if observed_weather.get('observations'):
            # Flatten to dataframe
            observed_weather_df = json_normalize(
                observed_weather.get('observations'))

        # Single-day observed
        else:
            # Flatten to dataframe
            observed_weather_df = json_normalize(observed_weather)

        # Set date as index
        observed_weather_df.set_index('date', inplace=True)

        # Drop unnecessary columns
        observed_weather_df.drop(
            columns=['_links.self.href'],
            axis=1, inplace=True)

        # Return dataframe
        return observed_weather_df

## Llevel 4: Sub-sub-sub-class - WeatherLocationForecast

In [10]:
class WeatherLocationForecast(WeatherLocation):
    # Class variables for clean_data() function
    # Main forecast
    coord_cols = ['longitude', 'latitude']

    drop_cols = [
        'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units', 'dewPoint.units'
    ]

    rename_map = {
        'startTime': 'start_time',
        'endTime': 'end_time',
        'conditionsCode': 'conditions_code',
        'conditionsText': 'conditions_text',
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.chance': 'precip_chance_%',
        'precipitation.amount': 'precip_amount_mm',
        'sky.cloudCover': 'sky_cloud_cover_%',
        'sky.sunshine': 'sky_sunshine_%',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.average': 'wind_avg_m_per_sec',
        'wind.max': 'wind_max_m_per_sec',
        'wind.min': 'wind_min_m_per_sec',
        'wind.bearing': 'wind_bearing_deg',
        'wind.direction': 'wind_direction_compass',
        'dewPoint.amount': 'dew_point_cels'
    }

    # Soil
    soil_coord_cols = ['longitude', 'latitude']

    soil_drop_cols = ['units']

    soil_rename_map = {
        'average_temp': 'soil_temp_avg_cels',
        'max_temp': 'soil_temp_max_cels',
        'min_temp': 'soil_temp_min_cels',
        'average_moisture': 'soil_moisture_avg_%',
        'max_moisture': 'soil_moisture_max_%',
        'min_moisture': 'soil_moisture_min_%',
    }

    def __init__(self, api_key, api_secret, latitude, longitude, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherLocationForecast, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.latitude = latitude
        self.longitude = longitude
        self.api_url = f"{self.api_url}/{self.latitude},{self.longitude}/forecasts"

    def get_data(self, start_day=None, end_day=None, offset=0, block_size=24):
        """
        Performs a HTTP GET request to obtain the 7-day forecast.

        Docs:
            http://developer.awhere.com/api/forecast-weather-api

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response: dict
            Dictionary containing the forecast.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self.api_url}?limit=10&offset={offset}&blockSize={block_size}"
        url_start_date = f"{self.api_url}/{start_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_end_date = f"{self.api_url}/{end_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_both_dates = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}&blockSize={block_size}"

        # Perform the HTTP request to obtain the Forecast for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return forecast
        return response.json()

    @staticmethod
    def extract_data(forecast):
        """Extract aWhere forecast data and returns
        it in a pandas dataframe.
        """
        # Initialize lists to store forecast
        forecast_main_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Extract the main forecast from the top-level dataframe
            forecast = row['forecast']

            # Faltten data into dataframe
            forecast_norm = json_normalize(forecast)

            # Drop soil moisture and soil temperature columns
            #  (will be extracted as indivdiual dataframes)
            forecast_norm.drop(columns=[
                'soilTemperatures',
                'soilMoisture',
            ],
                axis=1, inplace=True)
            # Assign date, lat/lon to dataframe
            forecast_norm['date'] = date
            forecast_norm['latitude'] = lat
            forecast_norm['longitude'] = lon

            # Set date as index
            forecast_norm.set_index(['date'], inplace=True)

            # Add the dataframe to a list of dataframes
            forecast_main_list.append(forecast_norm)

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_main_list)

    @staticmethod
    def extract_soil(forecast):
        """Extract aWhere forecast soil
        data and returns it in a pandas dataframe.
        """
        # Initialize lists to store soil dataframes
        forecast_soil_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Get soil temperature data
            forecast_soil_temp = row['forecast'][0].get('soilTemperatures')
            forecast_soil_moisture = row['forecast'][0].get('soilMoisture')

            # Flatten data into dataframe
            forecast_soil_temp_df = json_normalize(forecast_soil_temp)
            forecast_soil_moisture_df = json_normalize(forecast_soil_moisture)

            # Combine temperature and moisture
            forecast_soil_df = forecast_soil_temp_df.merge(
                forecast_soil_moisture_df, on='depth', suffixes=('_temp', '_moisture'))

            # Assign date, lat/lon to dataframe
            forecast_soil_df['date'] = date
            forecast_soil_df['latitude'] = lat
            forecast_soil_df['longitude'] = lon

            # Shorten depth values to numerics (will be used in MultiIndex)
            forecast_soil_df['depth'] = forecast_soil_df['depth'].apply(
                lambda x: x[0:-15])

            # Rename depth prior to indexing
            forecast_soil_df.rename(
                columns={'depth': 'ground_depth_m'}, inplace=True)

            # Create multi-index dataframe for date and soil depth (rename depth columns? rather long)
            soil_multi_index = forecast_soil_df.set_index(
                ['date', 'ground_depth_m'])

            # Add dataframe to list of dataframes
            forecast_soil_list.append(soil_multi_index)

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_soil_list)

    @classmethod
    def api_to_gdf(cls, api_object, forecast_type='main', kwargs=None):
        """
        forecast_type can either be 'main' or 'soil'.

        kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists

        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        if forecast_type.lower() == 'main':
            api_data_df = cls.extract_data(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.coord_cols,
                cls.drop_cols,
                cls.rename_map
            )

        elif forecast_type.lower() == 'soil': 
            api_data_df = cls.extract_soil(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.soil_coord_cols,
                cls.soil_drop_cols,
                cls.soil_rename_map
            )
        
        else: 
            raise ValueError("Invalid forecast type. Please choose 'main' or 'soil'.")

        return api_data_gdf

## Level 4: Sub-sub-sub-class - WeatherFieldNorms

In [11]:
class WeatherFieldNorms(WeatherField):

    # Class variables for clean_data() function 
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'location.fieldId', 'meanTemp.units', 'maxTemp.units',
        'minTemp.units', 'precipitation.units', 'solar.units',
        'dailyMaxWind.units', 'averageWind.units'
    ]

    rename_map = {
        'meanTemp.average': 'mean_temp_avg_cels',
        'meanTemp.stdDev': 'mean_temp_std_dev_cels',
        'maxTemp.average': 'max_temp_avg_cels',
        'maxTemp.stdDev': 'max_temp_std_dev_cels',
        'minTemp.average': 'min_temp_avg_cels',
        'minTemp.stdDev': 'min_temp_std_dev_cels',
        'precipitation.average': 'precip_avg_mm',
        'precipitation.stdDev': 'precip_std_dev_mm',
        'solar.average': 'solar_avg_w_h_per_m2',
        'minHumidity.average': 'min_humiduty_avg_%',
        'minHumidity.stdDev': 'min_humidity_std_dev_%',
        'maxHumidity.average': 'max_humiduty_avg_%',
        'maxHumidity.stdDev': 'max_humidity_std_dev_%',
        'dailyMaxWind.average': 'daily_max_wind_avg_m_per_sec',
        'dailyMaxWind.stdDev': 'daily_max_wind_std_dev_m_per_sec',
        'averageWind.average': 'average_wind_m_per_sec',
        'averageWind.stdDev': 'average_wind_std_dev_m_per_sec'
    }

    # Define field when intitializing class; no need to repeat for field
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherFieldNorms, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        self.api_url = f"{self.api_url}/{self.field_id}/norms"
        
    def get_data(self, start_day='01-01', end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 10-year historical norms.

        Docs:
            http://developer.awhere.com/api/reference/weather/norms

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the norms.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Perform the HTTP request to obtain the norms for the Field
        # Define URL variants
        url_single_day = f"{self.api_url}/{start_day}?limit=10&offset={offset}"
        url_multiple_days = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Get single day norms or date range
        response = requests.get(url_multiple_days, headers=auth_headers) if end_day else requests.get(
            url_single_day, headers=auth_headers)

        # Return the norms
        return response.json()

    @staticmethod
    def extract_data(historic_norms):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere historic norm data.

        Handles both single-day norms and multiple days.

        Parameters
        ----------
        historic_norms : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        historic_norms_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in norms
        if historic_norms.get('norms'):
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms.get('norms'))

        # Single-day norm
        else:
            # Flatten to dataframe
            historic_norms_df = json_normalize(historic_norms)

        # Set day as index
        historic_norms_df.set_index('day', inplace=True)

        # Drop unnecessary columns
        historic_norms_df.drop(
            columns=[
                '_links.self.href',
                '_links.curies',
                '_links.awhere:field.href'],
            axis=1, inplace=True)

        # Return dataframe
        return historic_norms_df

## Level 4: Sub-sub-sub-class - WeatherFieldObserved

In [12]:
class WeatherFieldObserved(WeatherField):

    # Class variables for clean_data() function     
    coord_cols = ['location.longitude', 'location.latitude']

    drop_cols = [
        'location.fieldId', 'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units'
    ]

    rename_map = {
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.amount': 'precip_amount_mm',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.morningMax': 'wind_morning_max_m_per_sec',
        'wind.dayMax': 'wind_day_max_m_per_sec',
        'wind.average': 'wind_avg_m_per_sec',
    }

    # Define field when intitializing class; no need to repeat for field
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherFieldObserved, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        self.api_url = f"{self.api_url}/{self.field_id}/observations"
        
    def get_data(self, start_day=None, end_day=None, offset=0):
        """
        Performs a HTTP GET request to obtain 7-day observed weather.

        Docs:
            http://developer.awhere.com/api/reference/weather/observations

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response : dict
            Dictionary containing the observed weather.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self.api_url}?limit=10&offset={offset}"
        url_start_date = f"{self.api_url}/{start_day}"
        url_end_date = f"{self.api_url}/{end_day}"
        url_both_dates = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}"

        # Perform the HTTP request to obtain the norms for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return the observed
        return response.json()
    
    @staticmethod
    def extract_data(observed_weather):
        """Creates a dataframe from a JSON-like
        dictionary of aWhere observed weather data.

        Parameters
        ----------
        observed_weather : dict
            aWhere historic norm data in dictionary format.

        Returns
        -------
        observed_weather_df : pandas dataframe
            Flattened dataframe version of historic norms.
        """
        # Check if multiple entries (days) are in observed
        if observed_weather.get('observations'):
            # Flatten to dataframe
            observed_weather_df = json_normalize(observed_weather.get('observations'))

        # Single-day observed
        else:
            # Flatten to dataframe
            observed_weather_df = json_normalize(observed_weather)

        # Set date as index
        observed_weather_df.set_index('date', inplace=True)

        # Drop unnecessary columns
        observed_weather_df.drop(
            columns=[
                '_links.self.href',
                '_links.curies',
                '_links.awhere:field.href'],
            axis=1, inplace=True)

        # Return dataframe
        return observed_weather_df

In [13]:
class WeatherFieldForecast(WeatherField):

    # Class variables for clean_data() function
    # Main forecast
    coord_cols = ['longitude', 'latitude']

    drop_cols = [
        'temperatures.units', 'precipitation.units',
        'solar.units', 'wind.units', 'dewPoint.units'
    ]

    rename_map = {
        'startTime': 'start_time',
        'endTime': 'end_time',
        'conditionsCode': 'conditions_code',
        'conditionsText': 'conditions_text',
        'temperatures.max': 'temp_max_cels',
        'temperatures.min': 'temp_min_cels',
        'precipitation.chance': 'precip_chance_%',
        'precipitation.amount': 'precip_amount_mm',
        'sky.cloudCover': 'sky_cloud_cover_%',
        'sky.sunshine': 'sky_sunshine_%',
        'solar.amount': 'solar_energy_w_h_per_m2',
        'relativeHumidity.average': 'rel_humidity_avg_%',
        'relativeHumidity.max': 'rel_humidity_max_%',
        'relativeHumidity.min': 'rel_humidity_min_%',
        'wind.average': 'wind_avg_m_per_sec',
        'wind.max': 'wind_max_m_per_sec',
        'wind.min': 'wind_min_m_per_sec',
        'wind.bearing': 'wind_bearing_deg',
        'wind.direction': 'wind_direction_compass',
        'dewPoint.amount': 'dew_point_cels'
    }

    # Soil
    soil_coord_cols = ['longitude', 'latitude']

    soil_drop_cols = ['units']

    soil_rename_map = {
        'average_temp': 'soil_temp_avg_cels',
        'max_temp': 'soil_temp_max_cels',
        'min_temp': 'soil_temp_min_cels',
        'average_moisture': 'soil_moisture_avg_%',
        'max_moisture': 'soil_moisture_max_%',
        'min_moisture': 'soil_moisture_min_%',
    }

    
    # Define field when intitializing class; no need to repeat for field
    #  in get_data() because it is already programmed into api_url
    def __init__(self, api_key, api_secret, field_id, base_64_encoded_secret_key=None,
                 auth_token=None, api_url=None):

        super(WeatherFieldForecast, self).__init__(
            api_key, api_secret, base_64_encoded_secret_key, auth_token, api_url)

        self.field_id = field_id
        self.api_url = f"{self.api_url}/{self.field_id}/forecasts"
        
    def get_data(self, start_day=None, end_day=None, offset=0, block_size=24):
        """
        Performs a HTTP GET request to obtain the 7-day forecast.

        Docs:
            http://developer.awhere.com/api/forecast-weather-api

        Parameters
        ----------
        field_id : str
            ID of the field.

        Returns
        -------
        response: dict
            Dictionary containing the forecast.

        Example
        -------
        """
        # Setup the HTTP request headers
        auth_headers = {
            "Authorization": f"Bearer {self.auth_token}"
        }

        # Define URL variants
        url_no_date = f"{self.api_url}?limit=10&offset={offset}&blockSize={block_size}"
        url_start_date = f"{self.api_url}/{start_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_end_date = f"{self.api_url}/{end_day}?limit=10&offset={offset}&blockSize={block_size}"
        url_both_dates = f"{self.api_url}/{start_day},{end_day}?limit=10&offset={offset}&blockSize={block_size}"

        # Perform the HTTP request to obtain the Forecast for the Field
        # Default - 7-day
        if not (start_day or end_day):
            response = requests.get(url_no_date, headers=auth_headers)

        # Single date - specify start day
        elif start_day and not end_day:
            response = requests.get(url_start_date, headers=auth_headers)

        # Single date - specify end day
        elif end_day and not start_day:
            response = requests.get(url_end_date, headers=auth_headers)

        # Date range
        elif start_day and end_day:
            response = requests.get(url_both_dates, headers=auth_headers)

        # Return forecast
        return response.json()
        
    @staticmethod
    def extract_data(forecast):
        """Extract aWhere forecast data and returns
        it in a pandas dataframe.
        """
        # Initialize lists to store forecast
        forecast_main_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Extract the main forecast from the top-level dataframe
            forecast = row['forecast']

            # Faltten data into dataframe
            forecast_norm = json_normalize(forecast)

            # Drop soil moisture and soil temperature columns
            #  (will be extracted as indivdiual dataframes)
            forecast_norm.drop(columns=[
                'soilTemperatures',
                'soilMoisture',
            ],
                axis=1, inplace=True)
            
            # Assign date, lat/lon to dataframe
            forecast_norm['date'] = date
            forecast_norm['latitude'] = lat
            forecast_norm['longitude'] = lon

            # Set date as index
            forecast_norm.set_index(['date'], inplace=True)

            # Add the dataframe to a list of dataframes
            forecast_main_list.append(forecast_norm)        

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_main_list)

    @staticmethod
    def extract_soil(forecast):
        """Extract aWhere forecast soil
        data and returns it in a pandas dataframe.
        """
        # Initialize lists to store soil dataframes
        forecast_soil_list = []

        # Check if more than one day
        if forecast.get('forecasts'):
            forecast_iterator = json_normalize(forecast.get('forecasts'))

        # Single day
        else:
            forecast_iterator = json_normalize(forecast)

        # Loop through each row in the top-level flattened dataframe
        for index, row in forecast_iterator.iterrows():

            # Extract date, lat, lon for insertion into lower-level dataframe outputs
            date = row['date']
            lat = row['location.latitude']
            lon = row['location.longitude']

            # Get soil temperature data
            forecast_soil_temp = row['forecast'][0].get('soilTemperatures')
            forecast_soil_moisture = row['forecast'][0].get('soilMoisture')

            # Flatten data into dataframe
            forecast_soil_temp_df = json_normalize(forecast_soil_temp)
            forecast_soil_moisture_df = json_normalize(forecast_soil_moisture)

            # Combine temperature and moisture
            forecast_soil_df = forecast_soil_temp_df.merge(
                forecast_soil_moisture_df, on='depth', suffixes=('_temp', '_moisture'))

            # Assign date, lat/lon to dataframe
            forecast_soil_df['date'] = date
            forecast_soil_df['latitude'] = lat
            forecast_soil_df['longitude'] = lon

            # Shorten depth values to numerics (will be used in MultiIndex)
            forecast_soil_df['depth'] = forecast_soil_df['depth'].apply(lambda x: x[0:-15])

            # Rename depth prior to indexing
            forecast_soil_df.rename(columns={'depth': 'ground_depth_m'}, inplace=True)

            # Create multi-index dataframe for date and soil depth (rename depth columns? rather long)
            soil_multi_index = forecast_soil_df.set_index(
                ['date', 'ground_depth_m'])

            # Add dataframe to list of dataframes
            forecast_soil_list.append(soil_multi_index)

        # Return merged lists of dataframes into a single dataframe
        return pd.concat(forecast_soil_list)

    @classmethod
    def api_to_gdf(cls, api_object, forecast_type='main', kwargs=None):
        """
        forecast_type can either be 'main' or 'soil'.

        kwargs is a dictionary that provides values beyond the default;
        unpack dictionary if it exists

        kwargs are the parameters to get_data() method

        kwargs={'start_day': '03-04', 'end_day': '03-07', 'offset': 2}
        """
        api_data_json = api_object.get_data(
            **kwargs) if kwargs else api_object.get_data()

        if forecast_type.lower() == 'main':
            api_data_df = cls.extract_data(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.coord_cols,
                cls.drop_cols,
                cls.rename_map
            )

        elif forecast_type.lower() == 'soil': 
            api_data_df = cls.extract_soil(api_data_json)

            api_data_gdf = cls.clean_data(
                api_data_df,
                cls.soil_coord_cols,
                cls.soil_drop_cols,
                cls.soil_rename_map
            )
        
        else: 
            raise ValueError("Invalid forecast type. Please choose 'main' or 'soil'.")

        return api_data_gdf

# Test/Demo Classes

Step 1 - Create aWhere object of desired class

Step 2 - Call the api_to_gdf() class method (with optional parameters) to go return geodataframe of the extracted and cleaned API data.

## Environment Setup

In [14]:
# Imports
import os

# Show all pandas columns
pd.set_option('max_columns', None)

# Define aWhere API key and secret
api_key = os.environ.get('AWHERE_API_KEY')
api_secret = os.environ.get('AWHERE_API_SECRET')

## Weather Norms - Location

In [15]:
# Define WeatherLocationNorms object, Bear Lake, RMNP, Colorado
norms_object = WeatherLocationNorms(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [16]:
# Define kwargs (parameters from get_data)
norms_kwargs = {'start_day': '05-01', 'end_day': '05-10'}

# Create geodataframe
norms_gdf = WeatherLocationNorms.api_to_gdf(norms_object, norms_kwargs)

In [17]:
# Display geodataframe
norms_gdf.head()

,mean_temp_avg_cels,mean_temp_std_dev_cels,max_temp_avg_cels,max_temp_std_dev_cels,min_temp_avg_cels,min_temp_std_dev_cels,precip_avg_mm,precip_std_dev_mm,solar_avg_w_h_per_m2,solar.stdDev,min_humiduty_avg_%,min_humidity_std_dev_%,max_humiduty_avg_%,max_humidity_std_dev_%,daily_max_wind_avg_m_per_sec,daily_max_wind_std_dev_m_per_sec,average_wind_m_per_sec,average_wind_std_dev_m_per_sec,geometry
day,,,,,,,,,,,,,,,,,,,
05-01,-1.1190,5.068417,3.876,5.916274,-6.114,4.420559,3.68035,9.554543,4431.667969,1377.411019,41.275,19.059746,87.468,12.352879,8.133095,1.840321,3.621491,0.984079,POINT (-105.64822 40.31325)
05-02,-0.1465,4.799937,6.008,5.381678,-6.301,5.016509,1.46045,2.810060,5611.167041,1472.916885,36.652,12.935928,88.573,7.825873,6.987793,1.524514,2.889469,0.687822,POINT (-105.64822 40.31325)
05-03,2.5060,3.589441,9.902,4.821597,-4.890,3.117275,1.58140,3.664253,6290.147144,1506.636049,28.086,15.302460,85.673,10.357255,7.615376,1.918557,3.427145,0.938786,POINT (-105.64822 40.31325)
05-04,4.3530,3.402978,11.472,5.071574,-2.766,2.787927,0.19920,0.629926,6444.413379,1520.595394,25.680,15.660493,74.959,16.532896,7.747343,3.441009,3.586217,1.860520,POINT (-105.64822 40.31325)
05-05,4.7560,2.985137,12.613,4.306329,-3.101,2.628673,0.23410,0.740289,6617.703345,1489.975061,23.204,17.430204,78.599,12.711234,7.259182,2.180336,3.262730,1.052785,POINT (-105.64822 40.31325)


## Weather Observed - Location

In [18]:
# Define WeatherLocationObserved object, Bear Lake, RMNP, Colorado
observed_object = WeatherLocationObserved(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [19]:
# Define kwargs (parameters from get_data)
#observed_kwargs = {'start_day': '05-01', 'end_day': '05-10'}

# Create geodataframe
observed_gdf = WeatherLocationObserved.api_to_gdf(observed_object)

In [20]:
# Display geodataframe
observed_gdf.head()

,temp_max_cels,temp_min_cels,precip_amount_mm,solar_energy_w_h_per_m2,rel_humidity_max_%,rel_humidity_min_%,wind_morning_max_m_per_sec,wind_day_max_m_per_sec,wind_avg_m_per_sec,geometry
date,,,,,,,,,,
2020-04-06,10.49,-5.43,0.0,6699.029785,60.000000,16.020000,6.063540,7.696202,3.357810,POINT (-105.64822 40.31325)
2020-04-07,9.87,-5.50,0.0,6736.287109,55.680000,16.980000,9.294910,10.221607,4.796959,POINT (-105.64822 40.31325)
2020-04-08,9.38,-5.51,0.0,6472.093262,69.620003,20.920000,5.814467,6.927653,2.706992,POINT (-105.64822 40.31325)
2020-04-09,8.65,-7.12,0.0,6298.243164,90.470001,28.090000,4.098253,5.974836,2.299960,POINT (-105.64822 40.31325)
2020-04-10,10.29,-4.51,0.0,5622.220703,89.510002,22.110001,4.583050,8.371593,3.077507,POINT (-105.64822 40.31325)


## Weather Forecast - Location

In [21]:
# Define WeatherLocationNorms object, Bear Lake, RMNP, Colorado
forecast_object = WeatherLocationForecast(
    api_key, api_secret, latitude=40.313250, longitude=-105.648222)

In [22]:
# Define kwargs (parameters from get_data)
#forecast_kwargs = {'start_day': '05-01', 'end_day': '05-10'}

# Create geodataframe - main forecast
forecast_main_gdf = WeatherLocationForecast.api_to_gdf(forecast_object, forecast_type='main')

In [23]:
# Display geodataframe
forecast_main_gdf.head()

,start_time,end_time,conditions_code,conditions_text,temp_max_cels,temp_min_cels,precip_chance_%,precip_amount_mm,sky_cloud_cover_%,sky_sunshine_%,solar_energy_w_h_per_m2,rel_humidity_avg_%,rel_humidity_max_%,rel_humidity_min_%,wind_avg_m_per_sec,wind_max_m_per_sec,wind_min_m_per_sec,wind_bearing_deg,wind_direction_compass,dew_point_cels,geometry
date,,,,,,,,,,,,,,,,,,,,,
2020-04-13,2020-04-13T00:00:00+00:00,2020-04-13T23:59:59+00:00,F31,"Cloudy, Moderate Rain, Light Wind/Calm",-8.088993,-16.826319,100.0,4.9375,98.500,1.500,6336.0,82.450000,99.199997,62.000000,3.567805,8.736952,0.955093,248.7,WSW,-14.547261,POINT (-105.64822 40.31325)
2020-04-14,2020-04-14T00:00:00+00:00,2020-04-14T23:59:59+00:00,E21,"Mostly Cloudy, Light Rain, Light Wind/Calm",-7.766667,-21.629675,100.0,2.2500,77.750,22.250,6372.0,84.570834,96.500000,71.599998,4.737279,10.411445,1.223272,289.5,WNW,-17.473278,POINT (-105.64822 40.31325)
2020-04-15,2020-04-15T00:00:00+00:00,2020-04-15T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-3.812199,-11.939640,100.0,10.0625,100.000,0.000,5268.0,92.166667,97.000000,82.800003,4.524884,6.853204,3.622969,259.8,W,-9.693567,POINT (-105.64822 40.31325)
2020-04-16,2020-04-16T00:00:00+00:00,2020-04-16T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-4.733228,-13.028670,100.0,24.1250,100.000,0.000,4902.0,95.199999,97.699997,91.400002,2.186328,3.313373,0.602639,139.2,SE,-9.255488,POINT (-105.64822 40.31325)
2020-04-17,2020-04-17T00:00:00+00:00,2020-04-17T23:59:59+00:00,C21,"Mostly Clear, Light Rain, Light Wind/Calm",-3.644154,-20.286654,100.0,1.6250,36.625,63.375,8100.0,91.604167,99.599998,79.800003,2.235362,4.156184,0.253718,254.1,WSW,-14.200592,POINT (-105.64822 40.31325)


In [24]:
# Create geodataframe - soil forecast
forecast_soil_gdf = WeatherLocationForecast.api_to_gdf(forecast_object, forecast_type='soil')

In [25]:
# Display geodataframe
forecast_soil_gdf.head()

soil_temp_avg_cels  soil_temp_max_cels  \
date       ground_depth_m                                           
2020-04-13 0-0.1                    -5.009957           -1.745410   
           0.1-0.4                  -1.554906           -1.550000   
           0.4-1                    -2.221712           -2.177637   
           1-2                      -1.246575           -1.238242   
2020-04-14 0-0.1                    -9.802844           -6.470451   

                           soil_temp_min_cels  soil_moisture_avg_%  \
date       ground_depth_m                                            
2020-04-13 0-0.1                    -6.715723            37.325000   
           0.1-0.4                  -1.558828            35.835968   
           0.4-1                    -2.267051            26.178707   
           1-2                      -1.248242            19.443127   
2020-04-14 0-0.1                   -11.950000            37.758334   

                           soil_moisture_max_%  soil_moisture_min_%  \
date       ground_depth_m                                             
2020-04-13 0-0.1                     37.500000            37.099998   
           0.1-0.4                   35.910542            35.700001   
           0.4-1                     26.203707            26.103708   
           1-2                       19.443127            19.443127   
2020-04-14 0-0.1                     37.900002            37.533333   

                                              geometry  
date       ground_depth_m                               
2020-04-13 0-0.1           POINT (-105.64822 40.31325)  
           0.1-0.4         POINT (-105.64822 40.31325)  
           0.4-1           POINT (-105.64822 40.31325)  
           1-2             POINT (-105.64822 40.31325)  
2020-04-14 0-0.1           POINT (-105.64822 40.31325)

## Weather Norms - Field

In [26]:
# Define WeatherFieldNorms object
norms_object = WeatherFieldNorms(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake')

In [27]:
# Define kwargs (parameters from get_data)
norms_kwargs = {'start_day': '05-01', 'end_day': '05-10'}

# Create geodataframe
norms_gdf = WeatherFieldNorms.api_to_gdf(norms_object, norms_kwargs)

In [28]:
# Display geodataframe
norms_gdf.head()

,mean_temp_avg_cels,mean_temp_std_dev_cels,max_temp_avg_cels,max_temp_std_dev_cels,min_temp_avg_cels,min_temp_std_dev_cels,precip_avg_mm,precip_std_dev_mm,solar_avg_w_h_per_m2,solar.stdDev,min_humiduty_avg_%,min_humidity_std_dev_%,max_humiduty_avg_%,max_humidity_std_dev_%,daily_max_wind_avg_m_per_sec,daily_max_wind_std_dev_m_per_sec,average_wind_m_per_sec,average_wind_std_dev_m_per_sec,geometry
day,,,,,,,,,,,,,,,,,,,
05-01,-1.1190,5.068417,3.876,5.916274,-6.114,4.420559,3.68035,9.554543,4431.667969,1377.411019,41.275,19.059746,87.468,12.352879,8.133095,1.840321,3.621491,0.984079,POINT (-105.64822 40.31325)
05-02,-0.1465,4.799937,6.008,5.381678,-6.301,5.016509,1.46045,2.810060,5611.167041,1472.916885,36.652,12.935928,88.573,7.825873,6.987793,1.524514,2.889469,0.687822,POINT (-105.64822 40.31325)
05-03,2.5060,3.589441,9.902,4.821597,-4.890,3.117275,1.58140,3.664253,6290.147144,1506.636049,28.086,15.302460,85.673,10.357255,7.615376,1.918557,3.427145,0.938786,POINT (-105.64822 40.31325)
05-04,4.3530,3.402978,11.472,5.071574,-2.766,2.787927,0.19920,0.629926,6444.413379,1520.595394,25.680,15.660493,74.959,16.532896,7.747343,3.441009,3.586217,1.860520,POINT (-105.64822 40.31325)
05-05,4.7560,2.985137,12.613,4.306329,-3.101,2.628673,0.23410,0.740289,6617.703345,1489.975061,23.204,17.430204,78.599,12.711234,7.259182,2.180336,3.262730,1.052785,POINT (-105.64822 40.31325)


## Weather Field - Observed

In [29]:
# Define WeatherFieldObserved object
observed_object = WeatherFieldObserved(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake')

In [30]:
# Define kwargs (parameters from get_data)
#observed_kwargs = {'start_day': '05-01', 'end_day': '05-10'}

# Create geodataframe
observed_gdf = WeatherFieldObserved.api_to_gdf(observed_object)

In [31]:
# Display geodataframe
observed_gdf.head()

,temp_max_cels,temp_min_cels,precip_amount_mm,solar_energy_w_h_per_m2,rel_humidity_max_%,rel_humidity_min_%,wind_morning_max_m_per_sec,wind_day_max_m_per_sec,wind_avg_m_per_sec,geometry
date,,,,,,,,,,
2020-04-06,10.49,-5.43,0.0,6699.029785,60.000000,16.020000,6.063540,7.696202,3.357810,POINT (-105.64822 40.31325)
2020-04-07,9.87,-5.50,0.0,6736.287109,55.680000,16.980000,9.294910,10.221607,4.796959,POINT (-105.64822 40.31325)
2020-04-08,9.38,-5.51,0.0,6472.093262,69.620003,20.920000,5.814467,6.927653,2.706992,POINT (-105.64822 40.31325)
2020-04-09,8.65,-7.12,0.0,6298.243164,90.470001,28.090000,4.098253,5.974836,2.299960,POINT (-105.64822 40.31325)
2020-04-10,10.29,-4.51,0.0,5622.220703,89.510002,22.110001,4.583050,8.371593,3.077507,POINT (-105.64822 40.31325)


## Weather Field - Forecast

In [32]:
# Define WeatherLocationForecast object, Bear Lake, RMNP, Colorado
forecast_object = WeatherFieldForecast(
    api_key, api_secret, field_id='CO-RMNP-Bear-Lake')

In [33]:
# Define kwargs (parameters from get_data)
#forecast_kwargs = {'start_day': '05-01', 'end_day': '05-10'}

# Create geodataframe - main forecast
forecast_main_gdf = WeatherFieldForecast.api_to_gdf(forecast_object, forecast_type='main')

In [34]:
# Display geodataframe
forecast_main_gdf.head()

,start_time,end_time,conditions_code,conditions_text,temp_max_cels,temp_min_cels,precip_chance_%,precip_amount_mm,sky_cloud_cover_%,sky_sunshine_%,solar_energy_w_h_per_m2,rel_humidity_avg_%,rel_humidity_max_%,rel_humidity_min_%,wind_avg_m_per_sec,wind_max_m_per_sec,wind_min_m_per_sec,wind_bearing_deg,wind_direction_compass,dew_point_cels,geometry
date,,,,,,,,,,,,,,,,,,,,,
2020-04-13,2020-04-13T00:00:00+00:00,2020-04-13T23:59:59+00:00,F31,"Cloudy, Moderate Rain, Light Wind/Calm",-8.088993,-16.826319,100.0,4.9375,98.500,1.500,6336.0,82.450000,99.199997,62.000000,3.567805,8.736952,0.955093,248.7,WSW,-14.547261,POINT (-105.64822 40.31325)
2020-04-14,2020-04-14T00:00:00+00:00,2020-04-14T23:59:59+00:00,E21,"Mostly Cloudy, Light Rain, Light Wind/Calm",-7.766667,-21.629675,100.0,2.2500,77.750,22.250,6372.0,84.570834,96.500000,71.599998,4.737279,10.411445,1.223272,289.5,WNW,-17.473278,POINT (-105.64822 40.31325)
2020-04-15,2020-04-15T00:00:00+00:00,2020-04-15T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-3.812199,-11.939640,100.0,10.0625,100.000,0.000,5268.0,92.166667,97.000000,82.800003,4.524884,6.853204,3.622969,259.8,W,-9.693567,POINT (-105.64822 40.31325)
2020-04-16,2020-04-16T00:00:00+00:00,2020-04-16T23:59:59+00:00,F41,"Cloudy, Heavy Rain, Light Wind/Calm",-4.733228,-13.028670,100.0,24.1250,100.000,0.000,4902.0,95.199999,97.699997,91.400002,2.186328,3.313373,0.602639,139.2,SE,-9.255488,POINT (-105.64822 40.31325)
2020-04-17,2020-04-17T00:00:00+00:00,2020-04-17T23:59:59+00:00,C21,"Mostly Clear, Light Rain, Light Wind/Calm",-3.644154,-20.286654,100.0,1.6250,36.625,63.375,8100.0,91.604167,99.599998,79.800003,2.235362,4.156184,0.253718,254.1,WSW,-14.200592,POINT (-105.64822 40.31325)


In [35]:
# Create geodataframe - soil forecast
forecast_soil_gdf = WeatherFieldForecast.api_to_gdf(forecast_object, forecast_type='soil')

In [36]:
# Display geodataframe
forecast_soil_gdf.head()

soil_temp_avg_cels  soil_temp_max_cels  \
date       ground_depth_m                                           
2020-04-13 0-0.1                    -5.009957           -1.745410   
           0.1-0.4                  -1.554906           -1.550000   
           0.4-1                    -2.221712           -2.177637   
           1-2                      -1.246575           -1.238242   
2020-04-14 0-0.1                    -9.802844           -6.470451   

                           soil_temp_min_cels  soil_moisture_avg_%  \
date       ground_depth_m                                            
2020-04-13 0-0.1                    -6.715723            37.325000   
           0.1-0.4                  -1.558828            35.835968   
           0.4-1                    -2.267051            26.178707   
           1-2                      -1.248242            19.443127   
2020-04-14 0-0.1                   -11.950000            37.758334   

                           soil_moisture_max_%  soil_moisture_min_%  \
date       ground_depth_m                                             
2020-04-13 0-0.1                     37.500000            37.099998   
           0.1-0.4                   35.910542            35.700001   
           0.4-1                     26.203707            26.103708   
           1-2                       19.443127            19.443127   
2020-04-14 0-0.1                     37.900002            37.533333   

                                              geometry  
date       ground_depth_m                               
2020-04-13 0-0.1           POINT (-105.64822 40.31325)  
           0.1-0.4         POINT (-105.64822 40.31325)  
           0.4-1           POINT (-105.64822 40.31325)  
           1-2             POINT (-105.64822 40.31325)  
2020-04-14 0-0.1           POINT (-105.64822 40.31325)

## Fields

In [37]:
# Create a Fields objects at the Fields class level
fields = Fields(api_key, api_secret)

In [38]:
# Show all fields associated with the api key & secret at the Fields clas level
fields.get()

You have 9 fields shown on this page.
#  Field Name 		 Field ID
-------------------------------------------
1. Manchester-VT 	 Manchester-VT
2. RMNP 	 Colorado-Test-1
3. RMNP 	 Colorado-Test-2
4. RMNP 	 Colorado-Test-4
5. RMNP 	 Colorado-Test-5
6. RMNP 	 RMNP-Test-Acre-1
7. RMNP 	 RMNP-Test-Acre-10000
8. Bear Lake 	 CO-RMNP-Bear-Lake
9. Bear Lake 	 Fields-Class-Test

Fields listed above.


In [39]:
# Show a single field (at the Fields class level)
fields.get('CO-RMNP-Bear-Lake')

{'name': 'Bear Lake',
 'acres': 1.0,
 'centerPoint': {'latitude': 40.31325, 'longitude': -105.648222},
 'farmId': 'Bear-Lake',
 'id': 'CO-RMNP-Bear-Lake',
 '_links': {'self': {'href': '/v2/fields/CO-RMNP-Bear-Lake'},
  'curies': [{'name': 'awhere',
    'href': 'http://awhere.com/rels/{rel}',
    'templated': True}],
  'awhere:observations': {'href': '/v2/weather/fields/CO-RMNP-Bear-Lake/observations'},
  'awhere:forecasts': {'href': '/v2/weather/fields/CO-RMNP-Bear-Lake/forecasts'},
  'awhere:plantings': {'href': '/v2/agronomics/fields/CO-RMNP-Bear-Lake/plantings'},
  'awhere:agronomics': {'href': '/v2/agronomics/fields/CO-RMNP-Bear-Lake/agronomicvalues'}}}

In [40]:
# Create field at the Fields class level
fields.create(field_id='N-VT', field_name='N-VT',
              center_latitude=42.5, center_longitude=-72.5,
              acres=1, farm_id='N-VT')

Attempting to create new field....



{'name': 'N-VT',
 'acres': 1.0,
 'centerPoint': {'latitude': 42.5, 'longitude': -72.5},
 'farmId': 'N-VT',
 'id': 'N-VT',
 '_links': {'self': {'href': '/v2/fields/N-VT'},
  'curies': [{'name': 'awhere',
    'href': 'http://awhere.com/rels/{rel}',
    'templated': True}],
  'awhere:observations': {'href': '/v2/weather/fields/N-VT/observations'},
  'awhere:forecasts': {'href': '/v2/weather/fields/N-VT/forecasts'},
  'awhere:plantings': {'href': '/v2/agronomics/fields/N-VT/plantings'},
  'awhere:agronomics': {'href': '/v2/agronomics/fields/N-VT/agronomicvalues'}}}

In [41]:
# Update the field name and farm id for a specific field (at the Fields class level)
fields.update(field_id='N-VT', name='New Field Name', farm_id='New Farm ID')

{'name': 'New Field Name',
 'acres': 1.0,
 'centerPoint': {'latitude': 42.5, 'longitude': -72.5},
 'farmId': 'New Farm ID',
 'id': 'N-VT',
 '_links': {'self': {'href': '/v2/fields/N-VT'},
  'curies': [{'name': 'awhere',
    'href': 'http://awhere.com/rels/{rel}',
    'templated': True}],
  'awhere:observations': {'href': '/v2/weather/fields/N-VT/observations'},
  'awhere:forecasts': {'href': '/v2/weather/fields/N-VT/forecasts'},
  'awhere:plantings': {'href': '/v2/agronomics/fields/N-VT/plantings'},
  'awhere:agronomics': {'href': '/v2/agronomics/fields/N-VT/agronomicvalues'}}}

In [42]:
# Delete a specific field at the Fields class level
fields.delete(field_id='N-VT')

Deleted field: N-VT


## Field

In [43]:
# Create field (from super class Fields)
Fields(api_key, api_secret).create(
    field_id='VT-Test-For-Delete', field_name='Test',
    center_latitude=42.5, center_longitude=-72.5,
    acres=1, farm_id='Test')

Attempting to create new field....



{'name': 'Test',
 'acres': 1.0,
 'centerPoint': {'latitude': 42.5, 'longitude': -72.5},
 'farmId': 'Test',
 'id': 'VT-Test-For-Delete',
 '_links': {'self': {'href': '/v2/fields/VT-Test-For-Delete'},
  'curies': [{'name': 'awhere',
    'href': 'http://awhere.com/rels/{rel}',
    'templated': True}],
  'awhere:observations': {'href': '/v2/weather/fields/VT-Test-For-Delete/observations'},
  'awhere:forecasts': {'href': '/v2/weather/fields/VT-Test-For-Delete/forecasts'},
  'awhere:plantings': {'href': '/v2/agronomics/fields/VT-Test-For-Delete/plantings'},
  'awhere:agronomics': {'href': '/v2/agronomics/fields/VT-Test-For-Delete/agronomicvalues'}}}

In [44]:
# Define a field a the Field subclass level
field = Field(api_key, api_secret, 'VT-Test-For-Delete')

In [45]:
# Get the field info at the Field subclass level
field.get()

{'name': 'Test',
 'acres': 1.0,
 'centerPoint': {'latitude': 42.5, 'longitude': -72.5},
 'farmId': 'Test',
 'id': 'VT-Test-For-Delete',
 '_links': {'self': {'href': '/v2/fields/VT-Test-For-Delete'},
  'curies': [{'name': 'awhere',
    'href': 'http://awhere.com/rels/{rel}',
    'templated': True}],
  'awhere:observations': {'href': '/v2/weather/fields/VT-Test-For-Delete/observations'},
  'awhere:forecasts': {'href': '/v2/weather/fields/VT-Test-For-Delete/forecasts'},
  'awhere:plantings': {'href': '/v2/agronomics/fields/VT-Test-For-Delete/plantings'},
  'awhere:agronomics': {'href': '/v2/agronomics/fields/VT-Test-For-Delete/agronomicvalues'}}}

In [46]:
# Update name and farm id at the Field subclass level
field.update(name='Test New Name', farm_id='Test Farm ID')

{'name': 'Test New Name',
 'acres': 1.0,
 'centerPoint': {'latitude': 42.5, 'longitude': -72.5},
 'farmId': 'Test Farm ID',
 'id': 'VT-Test-For-Delete',
 '_links': {'self': {'href': '/v2/fields/VT-Test-For-Delete'},
  'curies': [{'name': 'awhere',
    'href': 'http://awhere.com/rels/{rel}',
    'templated': True}],
  'awhere:observations': {'href': '/v2/weather/fields/VT-Test-For-Delete/observations'},
  'awhere:forecasts': {'href': '/v2/weather/fields/VT-Test-For-Delete/forecasts'},
  'awhere:plantings': {'href': '/v2/agronomics/fields/VT-Test-For-Delete/plantings'},
  'awhere:agronomics': {'href': '/v2/agronomics/fields/VT-Test-For-Delete/agronomicvalues'}}}

In [47]:
# Delete field at the Field subclass level
field.delete()

Deleted field: VT-Test-For-Delete
